## Common functions
The subroutines calculate the body proportion according to model, dataset and detected keypoints.

In [1]:
import math 

POSE_COCO_BODY_PARTS = {
    "Nose":0,"Neck":1,"RShoulder":2,"RElbow":3,"RWrist":4,"LShoulder":5,"LElbow":6,"LWrist":7,
    "RHip":8,"RKnee":9,"RAnkle":10,"LHip":11,"LKnee":12,"LAnkle":13,"REye":14,"LEye":15,"REar":16,"LEar":17,
    "Background":18
}

POSE_MPII_BODY_PARTS = {
    "Head":0,"Neck":1,"RShoulder":2,"RElbow":3,"RWrist":4,"LShoulder":5,"LElbow":6,"LWrist":7,
    "RHip":8,"RKnee":9,"RAnkle":10,"LHip":11,"LKnee":12,"LAnkle":13,"Chest":14,"Background":15
}

def all_presents(keypoints,model,*parts):
    for part in parts:
        if keypoints[model[part]][0] == 0 or keypoints[model[part]][0] == 0: return False
    return True

def body_part_length(keypoints,model,part1,part2):
    return math.sqrt(
        (keypoints[model[part1]][0]-keypoints[model[part2]][0])**2 + 
        (keypoints[model[part1]][1]-keypoints[model[part2]][1])**2 + 
        (keypoints[model[part1]][2]-keypoints[model[part2]][2])**2 ) \
            if all_presents(keypoints,model,part1,part2) else 0

## Coco dataset
def coco_head(keypoints):
    return body_part_length(keypoints,POSE_COCO_BODY_PARTS,'Nose','Neck')

def coco_rshoulder(keypoints):
    return body_part_length(keypoints,POSE_COCO_BODY_PARTS,'RShoulder','Neck')

def coco_lshoulder(keypoints):
    return body_part_length(keypoints,POSE_COCO_BODY_PARTS,'LShoulder','Neck')

def coco_avg_shoulder(keypoints):
    rs = coco_rshoulder(keypoints)
    ls = coco_lshoulder(keypoints)
    if rs > 0 and ls > 0:
        return (rs+ls)/2
    elif rs > 0 and ls <= 0:
        return rs
    else:
        return ls
    
def coco_body(keypoints):
    leg = max( 
        body_part_length(keypoints,POSE_COCO_BODY_PARTS,'LHip','LKnee')+
        body_part_length(keypoints,POSE_COCO_BODY_PARTS,'LKnee','LAnkle') \
        if all_presents(keypoints,POSE_COCO_BODY_PARTS,'LHip','LKnee','LAnkle') else 0,
        body_part_length(keypoints,POSE_COCO_BODY_PARTS,'RHip','RKnee')+
        body_part_length(keypoints,POSE_COCO_BODY_PARTS,'RKnee','RAnkle') \
        if all_presents(keypoints,POSE_COCO_BODY_PARTS,'RHip','RKnee','RAnkle') else 0)
    body = body_part_length(keypoints,POSE_COCO_BODY_PARTS,'Nose','Neck') + \
        max(body_part_length(keypoints,POSE_COCO_BODY_PARTS,'LHip','Neck'),body_part_length(keypoints,POSE_COCO_BODY_PARTS,'RHip','Neck')) + \
        leg 
    return (body if leg > 0 else 0)

def coco_corpus(keypoints):
    return max(body_part_length(keypoints,POSE_COCO_BODY_PARTS,'LHip','Neck'),
               body_part_length(keypoints,POSE_COCO_BODY_PARTS,'RHip','Neck'))

## MPII dataset
def mpi_head(keypoints):
    return body_part_length(keypoints,POSE_MPII_BODY_PARTS,'Head','Neck')

def mpi_rshoulder(keypoints):
    return body_part_length(keypoints,POSE_MPII_BODY_PARTS,'RShoulder','Neck')

def mpi_lshoulder(keypoints):
    return body_part_length(keypoints,POSE_MPII_BODY_PARTS,'LShoulder','Neck')

def mpi_avg_shoulder(keypoints):
    rs = mpi_rshoulder(keypoints)
    ls = mpi_lshoulder(keypoints)
    if rs > 0 and ls > 0:
        return (rs+ls)/2
    elif rs > 0 and ls <= 0:
        return rs
    else:
        return ls
    
def mpi_body(keypoints):
    leg = max( 
        body_part_length(keypoints,POSE_MPII_BODY_PARTS,'LHip','LKnee')+
        body_part_length(keypoints,POSE_MPII_BODY_PARTS,'LKnee','LAnkle') \
        if all_presents(keypoints,POSE_MPII_BODY_PARTS,'LHip','LKnee','LAnkle') else 0,
        body_part_length(keypoints,POSE_MPII_BODY_PARTS,'RHip','RKnee')+
        body_part_length(keypoints,POSE_MPII_BODY_PARTS,'RKnee','RAnkle') \
        if all_presents(keypoints,POSE_MPII_BODY_PARTS,'RHip','RKnee','RAnkle') else 0)
    body = body_part_length(keypoints,POSE_MPII_BODY_PARTS,'Head','Neck') + \
        max(body_part_length(keypoints,POSE_MPII_BODY_PARTS,'LHip','Neck'),body_part_length(keypoints,POSE_MPII_BODY_PARTS,'RHip','Neck')) + \
        leg 
    return (body if leg > 0 else 0)

def mpi_corpus(keypoints):
    return max(body_part_length(keypoints,POSE_COCO_BODY_PARTS,'LHip','Neck'),
               body_part_length(keypoints,POSE_COCO_BODY_PARTS,'RHip','Neck'))

## Building a proportion database by analyzing historical paintings with selected HPE architectures
The cell below processes the collection of historical painting by architecture built of the detector and algorithm. The result keypoints are associated to body parts and then basic proportions are calculated. Finally the result is saved to json file for future processing.

In [ ]:
from os import listdir
from os.path import isfile, join
from scipy.stats import shapiro, levene, bartlett, friedmanchisquare, kruskal
import json
import numpy as np

import cv2
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results)
from mmdet.apis import inference_detector, init_detector
local_runtime = False

try:
  from google.colab.patches import cv2_imshow  # for image visualization in colab
except:
  local_runtime = True


algoritms = {
    'Deeppose + Resnet on Mpii':
        {'pose_config':'configs/body/2d_kpt_sview_rgb_img/deeppose/mpii/res101_mpii_256x256.py',
         'pose_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/deeppose/deeppose_res101_mpii_256x256-87516a90_20210205.pth'
        },
    'Topdown Heatmap + Resnet on Mpii':
        {'pose_config':'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/mpii/res101_mpii_256x256.py',
         'pose_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/resnet/res101_mpii_256x256-416f5d71_20200812.pth'
        },
    'Topdown Heatmap + Scnet on Mpii':
        {'pose_config':'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/mpii/scnet101_mpii_256x256.py',
         'pose_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/scnet/scnet101_mpii_256x256-b4c2d184_20200812.pth'
        },
    'Topdown Heatmap + Resnetv1d on Mpii':
        {'pose_config':'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/mpii/resnetv1d152_mpii_256x256.py',
         'pose_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/resnetv1d/resnetv1d152_mpii_256x256-8b10a87c_20200812.pth'
        },
    'Topdown Heatmap + Seresnet on Mpii':
        {'pose_config':'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/mpii/seresnet101_mpii_256x256.py',
         'pose_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/seresnet/seresnet101_mpii_256x256-0ba14ff5_20200927.pth'
        },
    'Topdown Heatmap + Shufflenetv1 on Mpii':
        {'pose_config':'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/mpii/shufflenetv1_mpii_256x256.py',
         'pose_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/shufflenetv1/shufflenetv1_mpii_256x256-dcc1c896_20200925.pth'
        },
    'Topdown Heatmap + Hrnet + Dark on Mpii':
        {'pose_config':'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/mpii/hrnet_w48_mpii_256x256_dark.py',
         'pose_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_mpii_256x256_dark-0decd39f_20200927.pth'
        }, 
    'Topdown Heatmap + Resnext on Mpii':
        {'pose_config':'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/mpii/resnext152_mpii_256x256.py',
         'pose_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/resnext/resnext152_mpii_256x256-df302719_20200927.pth'
        }, 
    'Topdown Heatmap + Litehrnet on Mpii':
        {'pose_config':'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/mpii/litehrnet_30_mpii_256x256.py',
         'pose_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/litehrnet/litehrnet30_mpii_256x256-faae8bd8_20210622.pth'
        }, 
    'Topdown Heatmap + Shufflenetv2 on Mpii':
        {'pose_config':'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/mpii/shufflenetv2_mpii_256x256.py',
         'pose_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/shufflenetv2/shufflenetv2_mpii_256x256-4fb9df2d_20200925.pth'
        },  
    'Topdown Heatmap + Hrnet on Mpii':
        {'pose_config':'configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/mpii/hrnet_w48_mpii_256x256.py',
         'pose_checkpoint': 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_mpii_256x256-92cab7bd_20200812.pth'
        },       
    }

detectors = {
    'Faster-RCNN':
        {'det_config':'demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py',
         'det_checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
        },
    'Retina':
        {'det_config':'../mmdetection/configs/retinanet/retinanet_r50_fpn_mstrain_640-800_3x_coco.py',
         'det_checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/retinanet/retinanet_r50_fpn_mstrain_3x_coco/retinanet_r50_fpn_mstrain_3x_coco_20210718_220633-88476508.pth'
        },
    'Yolo':
        {'det_config':'../mmdetection/configs/yolo/yolov3_d53_mstrain-416_273e_coco.py',
         'det_checkpoint': 'https://download.openmmlab.com/mmdetection/v2.0/yolo/yolov3_d53_mstrain-416_273e_coco/yolov3_d53_mstrain-416_273e_coco-2b60fcd9.pth'
        },      
}

for det in detectors:
    for algo in algoritms:
        for threshold in [0.3,0.5,0.7]:
            print("%s %s %s" % (det,algo,threshold))
            # initialize pose model
            pose_model = init_pose_model(algoritms[algo]['pose_config'], algoritms[algo]['pose_checkpoint'])
            # initialize detector
            det_model = init_detector(detectors[det]['det_config'], detectors[det]['det_checkpoint'])

            path_to_process = ['gothic','renaissance','baroque','rococo','classicism','romantic']

            result = {}
            test_data = {}
            for path in path_to_process:
                fullpath = img = '../data/paintings/' + path
                imgfiles = [f for f in listdir(fullpath) if isfile(join(fullpath, f))]

                agw=[]
                agr=[]
                agb=[]

                for img in imgfiles:
                    mmdet_results = inference_detector(det_model, fullpath + "/" + img)
                    person_results = process_mmdet_results(mmdet_results, cat_id=1)
                    pose_results, returned_outputs = inference_top_down_pose_model(pose_model,
                       fullpath + "/" + img,
                       person_results,
                       bbox_thr=threshold,
                       format='xyxy',
                       dataset=pose_model.cfg.data.test.type) 
                    # calculate each person in picture
                    for pose in pose_results:
                        head = mpi_head(pose['keypoints'])
                        shoulder = mpi_avg_shoulder(pose['keypoints'])
                        body = mpi_body(pose['keypoints'])
                        corpus = mpi_corpus(pose['keypoints'])
                        gw = 0
                        gr = 0
                        gb = 0
                        if shoulder>0 and head>0:
                            gr = head/shoulder
                            agr.append(gr)
                        if body>0 and head>0: 
                            gw = head/body
                            agw.append(gw)
                        if corpus>0 and head>0:  
                            gb = head/corpus
                            agb.append(gb)

                test_data[path]={}
                test_data[path]['gr']=agr
                test_data[path]['gw']=agw
                test_data[path]['gb']=agb

            with open('json/'+det+'_'+algo+'_'+str(threshold)+'.json', 'w') as outfile:
                json.dump(test_data, outfile)

## Calculation of statistics for models
The cells below uses json files to calculate proper statistical significance tests.

### Kruskal–Wallis and Shapiro-Wilk

In [ ]:
from os import listdir
from os.path import isfile, join
from scipy.stats import shapiro, levene, bartlett, friedmanchisquare, kruskal
import json
import numpy as np

jpath = '../data/json/'

files = [f for f in listdir(jpath) if isfile(join(jpath, f))]

for f in files:
    styles = ['gothic','renaissance','baroque','rococo','classicism','romantic']

    data = json.load(open(join(jpath, f)))

    akruskal = {}
    for d in ['gb','gw','gr']:
        stat, p = kruskal(data['gothic'][d],data['renaissance'][d],data['baroque'][d],
            data['rococo'][d],data['classicism'][d],data['romantic'][d])
        if p > 0.05:
            akruskal[d] = '-'
            #print('populations are similar')
        else:
            #print('populations are different')
            akruskal[d] = '+'
        
    
    for style in styles:
        ashapiro = {}
        ile = {}
        for d in ['gb','gw','gr']:
            stat, p0 = shapiro(data[style][d])
            ile[d] = len(data[style][d])
            if p0 > 0.05:
                ashapiro[d] = '+'
                #print('probably Gaussian')
            else:
                #print('probably not Gaussian')
                ashapiro[d] = '-'
            
            
        print("%s;%s;%s;%s;%s;%s;%s;%s;%s;%s;%s;%s" % (f,style,d,ile['gb'],ile['gw'],ile['gr'],ashapiro['gb'],ashapiro['gw'],ashapiro['gr'],akruskal['gb'],akruskal['gw'],akruskal['gr']) )


### Standard deviation  

In [ ]:
from os import listdir
from os.path import isfile, join
from scipy.stats import shapiro, levene, bartlett, friedmanchisquare, kruskal
import json
import numpy as np

jpath = '../data/json/'

files = [f for f in listdir(jpath) if isfile(join(jpath, f))]

for f in files:
    styles = ['gothic','renaissance','baroque','rococo','classicism','romantic']

    data = json.load(open(join(jpath, f)))

    line = ''
    for style in styles:
        for d in ['gb','gw','gr']:
            line += ';' + str(np.std(data[style][d]))
            
    print("%s;%s" % (f,line ))


### Median 

In [ ]:
from os import listdir
from os.path import isfile, join
from scipy.stats import shapiro, levene, bartlett, friedmanchisquare, kruskal
import json
import numpy as np

jpath = '../data/json/'

files = [f for f in listdir(jpath) if isfile(join(jpath, f))]

for f in files:
    #print(f)
    styles = ['gothic','renaissance','baroque','rococo','classicism','romantic']

    data = json.load(open(join(jpath, f)))

    line = ''
    for style in styles:
        for d in ['gb','gw','gr']:
            line += ';' + str(np.median(data[style][d]))
            
    print("%s;%s" % (f,line ))


### posthoc Nemanyi Test for Faster-RCNN_Topdown Heatmap + Scnet on Mpii_0.3.json


In [ ]:
from os import listdir
from os.path import isfile, join
from scipy.stats import shapiro, levene, bartlett, friedmanchisquare, kruskal
import json
import numpy as np


from scikit_posthocs import posthoc_nemenyi_friedman, posthoc_nemenyi, posthoc_ttest, posthoc_dunn
from scipy.stats import shapiro, levene, bartlett, friedmanchisquare, kruskal, f_oneway, wilcoxon, mannwhitneyu

jfile = '../data/json/Faster-RCNN_Topdown Heatmap + Scnet on Mpii_0.3.json'
data = json.load(open(jfile))
styles = ['gothic','renaissance','baroque','rococo','classicism','romantic']
for style1 in styles:
    for style2 in styles:
        for d in ['gb','gw','gr']:
            print(d)
            x = [
                 data[style1][d],
                 data[style2][d]
                ]
            result =  sp.posthoc_nemenyi(x)
            print(result)

